In [1]:
import rdflib

In [2]:
#We're going to look for all the parents, grandparents, great-grandparents+ of the "Time series images" concept.
#For now, we'll filter on prefLabel, though it would probably be better to filter on URI.
tag = '\"Time series images\"'

In [3]:
# Create a graph from the vocabulary file.
g = rdflib.Graph().parse("product_vocabs/hlsp-ptype.rdf")

# Query the data in the graph g using SPARQL
q = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?ancestor
    WHERE {
        ?x rdf:type skos:Concept .
        ?x skos:prefLabel ?prefLabel .
        ?x skos:broader+ ?ancestor .
        FILTER (?prefLabel = """+tag+""") .
    }
"""
# Explanation:
# ?x rdf:type skos:Concept . #where variable ?x has the type <skos:Concept>
# ?x skos:prefLabel ?prefLabel . #define the variable ?prefLabel as the skos:prefLabel assigned to x
# ?x skos:broader+ ?ancestor . #define the variable ?ancestor as any concept transitively broader than variable ?x
# FILTER (?prefLabel = """+tag+""") . #Filter on the concept we selected using the python variable <tag>.

# The important thing here is the plus sign in "skos:broader+", which travels up the tree using SPARQL propery paths. 
# This is equivelant to finding all the skos:broaderTransitive relationships entailed by all the skos:broader relationships.
# If you instead used "skos:broader", you would only get the direct parents, and not the grandparents etc.
# Another alternative would be to use "skos:broader*", which is similar to "skos:broader+" but will also return the original concept that we filtered on.

# Apply the query to the graph and iterate through results
for r in g.query(q):
    print(r["ancestor"])
    
# The output is a list of all the concepts broader than <tag> ("Time series images"), all the way up the tree!

https://archive.stsci.edu/rdf/hlsp-ptype#Images
https://archive.stsci.edu/rdf/hlsp-ptype#Time_series
https://archive.stsci.edu/rdf/hlsp-ptype#Time_domain_data


Now we'll do the same thing, but for descendants instead of ancestors. Let's look up all descendents of the tag "Time series":

In [4]:
tag = '\"Time series\"'

In [5]:
g = rdflib.Graph().parse("product_vocabs/hlsp-ptype.rdf")

q = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?descendant
    WHERE {
        ?x rdf:type skos:Concept .
        ?x skos:prefLabel ?prefLabel .
        ?x ^skos:broader+ ?descendant .
        FILTER (?prefLabel = """+tag+""") .
    }
"""
# The only non-cosmetic change I made was changing "skos:broader+" to "^skos:broader+", which reverses the direction of the path being followed.
for r in g.query(q):
    print(r["descendant"])
    
# The output is a list of all the concepts narrower than <tag> ("Time series"), all the way down the tree!

https://archive.stsci.edu/rdf/hlsp-ptype#Light_curves
https://archive.stsci.edu/rdf/hlsp-ptype#Spectral_line_light_curves
https://archive.stsci.edu/rdf/hlsp-ptype#Time_series_images
https://archive.stsci.edu/rdf/hlsp-ptype#Time_series_spectra
https://archive.stsci.edu/rdf/hlsp-ptype#Velocity-time_curves
